In [1]:
import os
import pickle
import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB

import import_ipynb
from bohb_worker_1to3_5 import KerasWorker as worker

parser = argparse.ArgumentParser(description='bohb_server')
parser.add_argument('--min_budget',   type=float, help='Minimum number of epochs for training.',    default=50)
parser.add_argument('--max_budget',   type=float, help='Maximum number of epochs for training.',    default=200)
parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=100)
parser.add_argument('--worker', help='Flag to turn this into a worker process', action='store_true')
parser.add_argument('--run_id', type=str, help='A unique run id for this optimization run. An easy option is to use the job id of the clusters scheduler.')
parser.add_argument('--nic_name',type=str, help='Which network interface to use for communication.', default='lo')
parser.add_argument('--shared_directory',type=str, help='A directory that is accessible for all processes, e.g. a NFS share.', default='../BOHB_LAB_5CNN/result/5')

args=parser.parse_args(args=[])
    
# Every process has to lookup the hostname
host = hpns.nic_name_to_host(args.nic_name)


if args.worker:
    import time
    time.sleep(5)   # short artificial delay to make sure the nameserver is already running
    w = worker(run_id=args.run_id, host=host, timeout=120)
    w.load_nameserver_credentials(working_directory=args.shared_directory)
    w.run(background=False)
    exit(0)

# This example shows how to log live results. This is most useful
# for really long runs, where intermediate results could already be
# interesting. The core.result submodule contains the functionality to
# read the two generated files (results.json and configs.json) and
# create a Result object.
result_logger = hpres.json_result_logger(directory=args.shared_directory, overwrite=False)

# Start a nameserver:
NS = hpns.NameServer(run_id=args.run_id, host=host, port=0, working_directory=args.shared_directory)
ns_host, ns_port = NS.start()
# NS = hpns.NameServer(run_id=args.run_id, host='127.0.0.1', port=None, working_directory=args.shared_directory)
# ns_host, ns_port = NS.start()

# Start local worker
w = worker(run_id=args.run_id, host=host, nameserver=ns_host, nameserver_port=ns_port, timeout=120)
w.run(background=True)

# Run an optimizer
bohb = BOHB(  configspace = worker.get_configspace(),
                      run_id = args.run_id,
                      host=host,
                      nameserver=ns_host,
                      nameserver_port=ns_port,
                      result_logger=result_logger,
                      min_budget=args.min_budget, max_budget=args.max_budget,
               )
res = bohb.run(n_iterations=args.n_iterations)

# store results
with open(os.path.join(args.shared_directory, 'results.pkl'), 'wb') as fh:
    pickle.dump(res, fh)

# shutdown
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

importing Jupyter notebook from bohb_worker_1to3_5.ipynb
(11, 504960, 2)
(11, 101280, 2)


DEBUG:hpbandster:wait_for_workers trying to get the condition
INFO:hpbandster:DISPATCHER: started the 'discover_worker' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63414:WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7f6f5e5a7c88; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:33131>
INFO:hpbandster:DISPATCHER: started the 'job_runner' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63414:WORKER: No dispatcher found. Waiting for one to initiate contact.
INFO:hpbandster.run_None.worker.74c293875977.63414:WORKER: start listening for jobs
INFO:hpbandster:DISPATCHER: Pyro daemon running on 127.0.0.1:41215
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 0 currently in the pool.
INFO:hpbandster:DISPATCHER: discovered new worker, hpbandster.run_None.worker.74c293875977.63414140128903747392
DEBUG:hpbandster:HBMASTER: number of workers changed to 1
DEBUG:hpbandster:Enough workers to start this run!
DEBUG:hp

(11572, 480, 2)
(2321, 480, 2)


DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977.63414:WORKER: done with job (0, 0, 0), trying to register it.
INFO:hpbandster.run_None.worker.74c293875977.63414:WORKER: registered result for job (0, 0, 0) with dispatcher
DEBUG:hpbandster:DISPATCHER: job (0, 0, 0) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (0, 0, 0) on hpbandster.run_None.worker.74c293875977.63414140128903747392 finished
DEBUG:hpbandster:job_id: (0, 0, 0)
kwargs: {'config': {'dropout_rate': 0.2532997810949635, 'lr': 0.018779109790484905, 'lstm_units': 32, 'num_conv_layers': 5, 'num_fc_layers': 2, 'num_fc_units_1': 23, 'num_filters_1': 39, 'optimizer': 'SGD', 'num_fc_units_2': 30, 'num_filters_2': 10, 'num_filters_3': 11, 'num_filters_4': 22, 'num_filters_5': 17, 'sgd_momentum'

DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker 

...

DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker 

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker 

DEBUG:hpbandster:DISPATCHER: job (98, 0, 1) finished
INFO:hpbandster.run_None.worker.74c293875977.63414:WORKER: registered result for job (98, 0, 1) with dispatcher
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (98, 0, 1) on hpbandster.run_None.worker.74c293875977.63414140128903747392 finished
DEBUG:hpbandster:job_id: (98, 0, 1)
kwargs: {'config': {'dropout_rate': 0.3462563402715909, 'lr': 0.05042738178560933, 'lstm_units': 98, 'num_conv_layers': 5, 'num_fc_layers': 3, 'num_fc_units_1': 42, 'num_filters_1': 30, 'optimizer': 'Adam', 'num_fc_units_2': 175, 'num_fc_units_3': 43, 'num_filters_2': 63, 'num_filters_3': 29, 'num_filters_4': 5, 'num_filters_5': 15}, 'budget': 66.66666666666666, 'working_directory': '.'}
result: {'loss': 0.26330798864364624, 'info': {'test accuracy': 0.75, 'train accuracy': 0.7544360160827637, 'validation accuracy': 0.7366920113563538, 'number of parameters': 76116}}
exception: None

DEBUG:hpbandster:job_callback f

DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker 

DEBUG:hpbandster:job_callback for (99, 0, 0) started
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:job_callback for (99, 0, 0) got condition
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
DEBUG:hpbandster:done building a new model for budget 200.000000 based on 22/126 split
Best loss for this budget:0.051331





DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (99, 0, 0) finished
DEBUG:hpbandster:start sampling a new configuration.
DEBUG:hpbandster:best_vector: [0.45890741655748846, 0.6130855712692969, 0.6695503884409383, 0.7358062891677007, 0.6778922188842657, 0.8037195279306274, 0.7418609084089951, 0, 0.9725195120382676, 0.17725535689724453, 0.6458866553731191, 0.5749410967977017, 0.6284506808175216, 0.6812806212745313, 0.9478687754251326], 1.2849751202343744e-06, 134558.4913956031, 0.17290431365962114
DEBUG:hpbandster:done sampling a new configuration.
DEBUG:hpbandster:HBMA

In [2]:
# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()

print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in res.get_all_runs()])/args.max_budget))

Best found configuration: {'dropout_rate': 0.5001856856580251, 'lr': 0.0006140608741342609, 'lstm_units': 42, 'num_conv_layers': 2, 'num_fc_layers': 2, 'num_fc_units_1': 92, 'num_filters_1': 61, 'optimizer': 'Adam', 'num_fc_units_2': 77, 'num_filters_2': 150}
A total of 250 unique configurations where sampled.
A total of 300 runs where executed.
Total budget corresponds to 200.0 full function evaluations.
